## Introduction

Mr. Green has always wanted to have his own Cajun restaurant. He has been planning and now has the opportunity to open one in the city St. Louis, Missouri, USA. This city is a popular tourist location, and Mr. Green is wondering where would be the best location to open his restaurant. There are two locations open that he can choose from, Location 1 being near downtown and Location 2 being off the highway a few miles away. He has hired you, a Data Scientist, to look at these areas and advise him which location he should take.

Location 1 has the following address: 1234 Washington Ave, St. Louis, MO. 

Location 2 has the following address: 4366 Manchester Ave, St. Louis, MO.

## Data Section 

I will be using Foursquare Data of St Louis to analyze the two locations given by Mr.Green. The best location will be the most accessible, it should have more popular venues nearby, and it should be located at an ideal distance from it's competitors.

The venues near each location and their popularity will be generated by Foursquare API.
I can also check what kind of venues they are- the type of venues is important to know.

I can also use FourSquare to check trending venues near both locations at lunch or dinner time to assess how popular the nearby venues are.

I will also search for other restaurants of the same category in town by a Foursquare search query. These are the competitor restaurants, and knowing where they are can help determine the better location.

## Methodology

To begin exploring the area, we will download the necessary libraries and programs:

In [1]:
import requests 
import pandas as pd 
import numpy as np 
import random 

!pip install geopy
from geopy.geocoders import Nominatim 

from IPython.display import Image 
from IPython.core.display import HTML 
    
from pandas.io.json import json_normalize

! pip install folium==0.5.0
import folium 

print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Folium installed
Libraries imported.


And to use FourSquare, let's set up the credentials:

In [2]:
#cell is hidden with Foursquare Credentials

CLIENT_ID = 'JVKBDMEUZ1WV343SRX0QJQ35FTSXCGZ3ESHQFGZTR5AR5LCW' 
CLIENT_SECRET = 'WSI0IRNHJZQWCSNUDATW3COJZW1GPTNK3YGJG00PD3UAKPQG' 
ACCESS_TOKEN = 'ALBKFLO3DC50SI4IY100JNOQSU0I33U20GIRCHQOKPOTSOER' 
VERSION = '20180605' 
LIMIT = 100 


First we will get Mr Green's two possible locations' coordinates so we can use them through our study:

In [3]:
address1 = '1234 Washington Ave, St. Louis, MO'

geolocator = Nominatim(user_agent="foursquare_agent")
location1 = geolocator.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print("Location 1 coordinates:")
print(latitude1, longitude1)

address2 = '4366 Manchester Ave, St. Louis, MO'

geolocator = Nominatim(user_agent="foursquare_agent")
location2 = geolocator.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print("Location 2 coordinates")
print(latitude2, longitude2)


Location 1 coordinates:
38.631805172357204 -90.19730635112238
Location 2 coordinates
38.62647895 -90.25927867157164


Now we will compare the number of venues by each Location, starting with Location 1:

Top 100 venues within a 1 mile radius of Location 1:

In [4]:
LIMIT= 100
radius =1609

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude1, 
    longitude1, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=JVKBDMEUZ1WV343SRX0QJQ35FTSXCGZ3ESHQFGZTR5AR5LCW&client_secret=WSI0IRNHJZQWCSNUDATW3COJZW1GPTNK3YGJG00PD3UAKPQG&v=20180605&ll=38.631805172357204,-90.19730635112238&radius=1609&limit=100'

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60bfa5adb1b0ae09f26c5fcd'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown West',
  'headerFullLocation': 'Downtown West, St Louis',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 133,
  'suggestedBounds': {'ne': {'lat': 38.64628618683822,
    'lng': -90.17880346168577},
   'sw': {'lat': 38.61732415787619, 'lng': -90.21580924055898}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '553c1aba498e5c63a1ecc2bd',
       'name': 'Star Clipper',
       'location': {'address': '1319 Washington Ave',
        'lat': 38.632414287962106,
        'lng': -90.19908137314481,
        'labeledLa

In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    Cleaning the json file:

In [7]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Star Clipper,Comic Shop,38.632414,-90.199081
1,Rooster,Breakfast Spot,38.630186,-90.195476
2,Washington Ave Post,Coffee Shop,38.632390,-90.198981
3,City Museum,Museum,38.633463,-90.200989
4,Bridge Tap House & Wine Bar,Wine Bar,38.629737,-90.194590


We now have a dataframe containing up to 100 venues within 1 mile of Location 1. Let's do the same with Location 2:

Top 100 venues within a 1 mile radius of Location 2:

In [8]:
LIMIT= 100
radius = 1609

url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude2, 
    longitude2, 
    radius, 
    LIMIT)
url2

'https://api.foursquare.com/v2/venues/explore?&client_id=JVKBDMEUZ1WV343SRX0QJQ35FTSXCGZ3ESHQFGZTR5AR5LCW&client_secret=WSI0IRNHJZQWCSNUDATW3COJZW1GPTNK3YGJG00PD3UAKPQG&v=20180605&ll=38.62647895,-90.25927867157164&radius=1609&limit=100'

In [9]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '60bfa5c3318dee44c7be9759'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Forest Park Southeast',
  'headerFullLocation': 'Forest Park Southeast, St Louis',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 108,
  'suggestedBounds': {'ne': {'lat': 38.64095996448101,
    'lng': -90.24077715659945},
   'sw': {'lat': 38.611997935518986, 'lng': -90.27778018654382}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5801720fd67cc1f3eb516a9b',
       'name': 'Sauce on the Side',
       'location': {'address': 'Manchester Avenue',
        'crossStreet': 'Tower Grove',
        'lat': 38.627133942381

    Cleaning the json file:

In [10]:
venues2 = results2['response']['groups'][0]['items']
    
nearby_venues2 = json_normalize(venues2) # flatten JSON

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues2 =nearby_venues2.loc[:, filtered_columns]

nearby_venues2['venue.categories'] = nearby_venues2.apply(get_category_type, axis=1)

nearby_venues2.columns = [col.split(".")[-1] for col in nearby_venues2.columns]

nearby_venues2.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Sauce on the Side,Pizza Place,38.627134,-90.256722
1,Urban Chestnut Brewing Company,Brewery,38.626629,-90.261139
2,The Gramophone,Bar,38.627205,-90.255979
3,The URB,Brewery,38.626502,-90.261784
4,Layla,New American Restaurant,38.627023,-90.257504


We now have a dataframe containing up to 100 venues within 1 mile of Location 2. Let's compare how many venues were returned were returned for each Location:

In [11]:
print('{} venues were returned by Foursquare for Location 1.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare for Location 1.


In [12]:
print('{} venues were returned by Foursquare for Location 2.'.format(nearby_venues2.shape[0]))

100 venues were returned by Foursquare for Location 2.


Let's map these venues on a map to get an idea of their proximity to the Locations, starting with Location 1:

In [13]:
venues_map = folium.Map(location=[latitude1, longitude1], zoom_start=15) 


folium.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    popup='Location 1',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)



for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

venues_map

Map of Location 2 venues:

In [14]:
venues2_map = folium.Map(location=[latitude2, longitude2], zoom_start=15) 



folium.CircleMarker(
    [latitude2, longitude2],
    radius=10,
    popup='Location 2',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues2_map)



for lat, lng, label in zip(nearby_venues2.lat, nearby_venues2.lng, nearby_venues2.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues2_map)

venues2_map

Let's compare trending venues at Lunch and Dinner for both locations to assess possible foot traffic:

In [15]:
urlt = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION)

results1 = requests.get(urlt).json()
results1

{'meta': {'code': 200, 'requestId': '60bfa657dae81048464c5dcd'},
 'response': {'venues': []}}

In [16]:
if len(results1['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results1['response']['venues']
    trending_venues_df = pd.json_normalize(trending_venues)

    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

Trending Venues by Location 1 at 10 am:

In [23]:
trending_venues_df

'No trending venues are available at the moment!'

Trending Venues by Location 1 at 12pm:

In [17]:
# run at 12pm
trending_venues_df

'No trending venues are available at the moment!'

Let's map those venues along with Location 1:

In [18]:
if len(results1['response']['venues']) == 0:
    venues_map_trending = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map_trending = folium.Map(location=[latitude1, longitude1], zoom_start=15) # generate map centered around Location 1


    # add Location 1 as a red circle mark
    folium.CircleMarker(
        [latitude1, longitude1],
        radius=10,
        popup='Location 1',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map_trending)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map_trending)

In [19]:
venues_map_trending

'Cannot generate visual as no trending venues are available at the moment!'

Let's check trending venues for Location 2:

In [20]:
# define URL
urlt2 = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION)

# send GET request and get trending venues
results2 = requests.get(urlt2).json()
results2

{'meta': {'code': 200, 'requestId': '60bfa66e7bdef64dd7ea4eda'},
 'response': {'venues': []}}

In [21]:
if len(results2['response']['venues']) == 0:
    trending_venues_df2 = 'No trending venues are available at the moment!'
    
else:
    trending_venues2 = results2['response']['venues']
    trending_venues_df2 = json_normalize(trending_venues2)

    # filter columns
    columns_filtered2 = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df2 = trending_venues_df2.loc[:, columns_filtered2]

    # filter the category for each row
    trending_venues_df2['categories'] = trending_venues_df2.apply(get_category_type, axis=1)

Trending venues at 10am:

In [26]:
trending_venues_df2

'No trending venues are available at the moment!'

Trending venues at 12pm:

In [22]:
trending_venues_df2

'No trending venues are available at the moment!'

Let's generate a map of Trending venues by Location 2:

In [23]:
if len(results2['response']['venues']) == 0:
    venues2_map_trending = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues2_map_trending = folium.Map(location=[latitude2, longitude2], zoom_start=15) 


  
    folium.CircleMarker(
        [latitude2, longitude2],
        radius=10,
        popup='Location 2',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues2_map_trending)


    for lat, lng, label in zip(trending_venues_df2['location.lat'], trending_venues_df2['location.lng'], trending_venues_df2['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues2_map_trending)

In [24]:
venues2_map_trending

'Cannot generate visual as no trending venues are available at the moment!'

Let's analyze the two Locations' venues to see the type and prevalence of venues, starting with Location 1:

In [29]:
pd.set_option('display.max_rows', 100)
venue1_counts = nearby_venues['categories'].value_counts()
venue1_counts

Hotel                       9
Coffee Shop                 5
American Restaurant         5
BBQ Joint                   4
Italian Restaurant          3
Steakhouse                  3
Bar                         3
Brewery                     3
Beer Garden                 2
Park                        2
Gift Shop                   2
Sports Bar                  2
Baseball Stadium            2
Sushi Restaurant            2
Hockey Arena                2
Museum                      2
Lounge                      2
Scenic Lookout              2
Pub                         2
Diner                       2
Food Truck                  1
Frozen Yogurt Shop          1
ATM                         1
Mediterranean Restaurant    1
Sandwich Place              1
Thai Restaurant             1
Speakeasy                   1
Aquarium                    1
Burger Joint                1
Comic Shop                  1
Smoke Shop                  1
Nightclub                   1
Spa                         1
Restaurant

Let's convert this list to a dataframe to see how many restaurants are included.

In [41]:
df = nearby_venues['categories'].value_counts().rename_axis('Venue').reset_index(name='Counts')
df

,Venue,Counts
0,Hotel,10
1,Coffee Shop,5
2,American Restaurant,5
3,BBQ Joint,4
4,Italian Restaurant,3
...,...,...
56,Plaza,1
57,Historic Site,1
58,Food Truck,1
59,New American Restaurant,1


In [46]:
df.Venue.str.count("Restaurant").sum()

11

So there are 11 restaurants by Location 1.

Let's do the same for Location 2:

In [30]:
pd.set_option('display.max_rows', 100)
venue2_counts = nearby_venues2['categories'].value_counts()
venue2_counts

New American Restaurant            4
Brewery                            4
Sandwich Place                     4
Wine Bar                           4
Gay Bar                            4
Pub                                3
Pizza Place                        3
Cocktail Bar                       3
Coffee Shop                        3
Café                               3
Bar                                3
Hotel                              2
Fast Food Restaurant               2
Storage Facility                   2
Chinese Restaurant                 2
Donut Shop                         2
Italian Restaurant                 2
Mediterranean Restaurant           2
Fried Chicken Joint                2
Dessert Shop                       2
Rock Club                          2
Bakery                             2
Bagel Shop                         1
Antique Shop                       1
Beer Bar                           1
BBQ Joint                          1
Botanical Garden                   1
G

In [47]:
df2 = nearby_venues2['categories'].value_counts().rename_axis('Venue').reset_index(name='Counts')
df2

,Venue,Counts
0,New American Restaurant,4
1,Wine Bar,4
2,Sandwich Place,4
3,Gay Bar,4
4,Brewery,4
...,...,...
56,Dive Bar,1
57,French Restaurant,1
58,Mediterranean Restaurant,1
59,Poke Place,1


In [48]:
df2.Venue.str.count("Restaurant").sum()

12

Let's see where are the other Cajun Restaurants within 2 miles of each Location to see where the competitors are located.
Starting with Location 1:

In [49]:
search_query = 'Cajun'
radius = 3218
print(search_query + ' .... OK!')

Cajun .... OK!


In [50]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=JVKBDMEUZ1WV343SRX0QJQ35FTSXCGZ3ESHQFGZTR5AR5LCW&client_secret=WSI0IRNHJZQWCSNUDATW3COJZW1GPTNK3YGJG00PD3UAKPQG&ll=38.631805172357204,-90.19730635112238&oauth_token=ALBKFLO3DC50SI4IY100JNOQSU0I33U20GIRCHQOKPOTSOER&v=20180605&query=Cajun&radius=3218&limit=100'

In [51]:
results3 = requests.get(url).json()
results3

{'meta': {'code': 200, 'requestId': '60bfab5b7bdef64dd7f840d6'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '510419b7e4b05cc1bed9b2b0',
    'name': 'Crystal Cajun Cook-Off',
    'location': {'address': 'City Hall',
     'lat': 38.625822,
     'lng': -90.19964,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.625822,
       'lng': -90.19964}],
     'distance': 696,
     'postalCode': '63103',
     'cc': 'US',
     'city': 'St Louis',
     'state': 'MO',
     'country': 'United States',
     'formattedAddress': ['City Hall', 'St Louis, MO 63103']},
    'categories': [{'id': '4bf58dd8d48988d17a941735',
      'name': 'Cajun / Creole Restaurant',
      'pluralName': 'Cajun / Creole Restaurants',
      'shortName': 'Cajun / Creole',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cajun_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1623173979',
    'hasPerk': Fals

In [52]:
venues3 = results3['response']['venues']

dataframe1 = pd.json_normalize(venues3)
dataframe1

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,510419b7e4b05cc1bed9b2b0,Crystal Cajun Cook-Off,"[{'id': '4bf58dd8d48988d17a941735', 'name': 'C...",v-1623173979,False,City Hall,38.625822,-90.199640,"[{'label': 'display', 'lat': 38.625822, 'lng':...",696,63103,US,St Louis,MO,United States,"[City Hall, St Louis, MO 63103]",NaN
1,608137e7abc301770ff4cb31,Soulard's Cajun Corner,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1623173979,False,730 Carroll Street,38.610649,-90.201060,"[{'label': 'display', 'lat': 38.6106494, 'lng'...",2377,63104,US,St Louis,MO,United States,"[730 Carroll Street, St Louis, MO 63104]",1367489617
2,587faa7c0bc55b4fc31c60c1,Norton's Cajun Corner,"[{'id': '4bf58dd8d48988d17a941735', 'name': 'C...",v-1623173979,False,730 Carroll St,38.610432,-90.201595,"[{'label': 'display', 'lat': 38.61043215866372...",2408,63104,US,St Louis,MO,United States,"[730 Carroll St, St Louis, MO 63104]",NaN


    Let's clean up the resulting data:

In [53]:
filtered_columns1 = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe1_filtered = dataframe1.loc[:, filtered_columns1]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe1_filtered['categories'] = dataframe1_filtered.apply(get_category_type, axis=1)

dataframe1_filtered.columns = [column.split('.')[-1] for column in dataframe1_filtered.columns]

dataframe1_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Crystal Cajun Cook-Off,Cajun / Creole Restaurant,City Hall,38.625822,-90.199640,"[{'label': 'display', 'lat': 38.625822, 'lng':...",696,63103,US,St Louis,MO,United States,"[City Hall, St Louis, MO 63103]",510419b7e4b05cc1bed9b2b0
1,Soulard's Cajun Corner,Restaurant,730 Carroll Street,38.610649,-90.201060,"[{'label': 'display', 'lat': 38.6106494, 'lng'...",2377,63104,US,St Louis,MO,United States,"[730 Carroll Street, St Louis, MO 63104]",608137e7abc301770ff4cb31
2,Norton's Cajun Corner,Cajun / Creole Restaurant,730 Carroll St,38.610432,-90.201595,"[{'label': 'display', 'lat': 38.61043215866372...",2408,63104,US,St Louis,MO,United States,"[730 Carroll St, St Louis, MO 63104]",587faa7c0bc55b4fc31c60c1


Let's visualize these locations:

In [54]:
dataframe1_filtered.name

0    Crystal Cajun Cook-Off
1    Soulard's Cajun Corner
2     Norton's Cajun Corner
Name: name, dtype: object

In [55]:
venues1_map = folium.Map(location=[latitude1, longitude1], zoom_start=13) 

folium.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    color='red',
    popup='Location 1',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues1_map)


folium.CircleMarker(
    [latitude2, longitude2],
    radius=10,
    color='green',
    popup='Location 2',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues1_map)


for lat, lng, label in zip(dataframe1_filtered.lat, dataframe1_filtered.lng, dataframe1_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues1_map)

# display map
venues1_map

Let's now see where the competitors are in relation to Location 2:

In [56]:
search_query = 'Cajun'
radius = 5000
print(search_query + ' .... OK!')

url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url2

results4 = requests.get(url2).json()
results4

Cajun .... OK!


{'meta': {'code': 200, 'requestId': '60bfab6fdf48c7065c59dfc4'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '510419b7e4b05cc1bed9b2b0',
    'name': 'Crystal Cajun Cook-Off',
    'location': {'address': 'City Hall',
     'lat': 38.625822,
     'lng': -90.19964,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.625822,
       'lng': -90.19964}],
     'distance': 5187,
     'postalCode': '63103',
     'cc': 'US',
     'city': 'St Louis',
     'state': 'MO',
     'country': 'United States',
     'formattedAddress': ['City Hall', 'St Louis, MO 63103']},
    'categories': [{'id': '4bf58dd8d48988d17a941735',
      'name': 'Cajun / Creole Restaurant',
      'pluralName': 'Cajun / Creole Restaurants',
      'shortName': 'Cajun / Creole',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cajun_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1623173999',
    'hasPerk': Fal

In [57]:
venues4 = results4['response']['venues']

dataframe2 = pd.json_normalize(venues4)
dataframe2.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,510419b7e4b05cc1bed9b2b0,Crystal Cajun Cook-Off,"[{'id': '4bf58dd8d48988d17a941735', 'name': 'C...",v-1623173999,False,City Hall,38.625822,-90.19964,"[{'label': 'display', 'lat': 38.625822, 'lng':...",5187,63103,US,St Louis,MO,United States,"[City Hall, St Louis, MO 63103]"


    Cleaning up the data:

In [58]:
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe2_filtered = dataframe2.loc[:, filtered_columns1]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe2_filtered['categories'] = dataframe2_filtered.apply(get_category_type, axis=1)

dataframe2_filtered.columns = [column.split('.')[-1] for column in dataframe2_filtered.columns]

dataframe2_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Crystal Cajun Cook-Off,Cajun / Creole Restaurant,City Hall,38.625822,-90.19964,"[{'label': 'display', 'lat': 38.625822, 'lng':...",5187,63103,US,St Louis,MO,United States,"[City Hall, St Louis, MO 63103]",510419b7e4b05cc1bed9b2b0


Let's visualize these results:

In [59]:
dataframe2_filtered.name

0    Crystal Cajun Cook-Off
Name: name, dtype: object

In [60]:
venues_map2 = folium.Map(location=[latitude2, longitude2], zoom_start=13) 

folium.CircleMarker(
    [latitude2, longitude2],
    radius=10,
    color='green',
    popup='Location 2',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map2)


folium.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    color='red',
    popup='Location 1',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map2)

for lat, lng, label in zip(dataframe2_filtered.lat, dataframe2_filtered.lng, dataframe2_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map2)

# display map
venues_map2

## Results and Discussion

Let's analyze the findings one by one. First we compared the number of venues within 1 mile of Location 1 versus by Location 2. Both Locations had 100 venues nearby. It is possible that there are more venues, but because we had a limit of 100 the total number stopped at 100.  Regardless, both Locations have a similar amount of venues in their reach, and so would likely attract a similar amount of people from those venues.

We then mapped these venues and our Locations. According to the first map, the venues by Location 1 are centralized to downtown St. Louis and the east side of the city. The proximity of the venues to Location 1 also suggests that it is possible to access Location 1 by walking or by a short car ride(depending on the venue). Looking at the Location 2's map, there are some venues in the immediate vicinity of Location 2, accessible by foot, but a majority of the venues are more spread through the center of the city. The distance from these venues could be a walk or short car ride.

In the immediate vicinity of Location 1 are some bars, restaurants, and coffee shops. In the immediate vicinity of Location 2 are also restaurants, bars, and coffee shops. 

We see that because Location 2 is closer to the center of the city, it is more accessible to the North side, West Side, East Side and South side of St. Louis, whereas Location one is more accessible to people who are downtown or on the East side. This means that Location 2 is more accessible to not only tourists driving through different parts of the city, but also to a majority of the neighborhoods in St Louis.

After the maps, we attempted to view trending venues at the times of 10am and 12pm using Foursquare API, however no venues were appearing. So instead we looked at the lists of 100 venues from the first step and assessed what types of venues were present.

Looking at the Location 1 list of venues, the top 5 most common venues were Hotels (9), Coffee Shops (5), New American Restaurants (5), Barbeque Restaurants (4), and Italian Restaurants (3). There are a total of 11 restaurants out of the 100 venues. The main attractions in this list include an aqaurium, museum, baseball stadium, national park, theme park, and football stadium.
 
Looking at the Location 2 list of venues, the top 5 most common venues were New American Restaurants (4), Coffee Shops(4), Sandwich Shops (4), Bars (8), and Pubs (3). Out of the 100 venues, there are a total of 12 restaurants. The main attractions in this list include gardens, parks, an ice skating rink, baseball park, a planetarium, and a museum. There are also schools and stores nearby.

While both locations had a similar number of restaurants, Location 1 had more similar types of restaurants and Location 2 has more variety of restaurants. So if tourists or residents are looking for different types of food, Location 2 may be ideal for a Cajun restaurant. Additionally Location 2's most common venues include Bars and Pubs, which are commonly frequented afternoon through evening, which are the same hours where people eat food- this could mean a lot of foot traffic to the restaurant if people are hungry. However Location 1 does have more Hotels, and tourists may prefer eating somewhere closer to where they are staying.
They both have a good amount of nearby tourist attractions, but Location 2 also has schools and stores nearby, making it more likely to be a regular stop for the many residents of the city.

Lastly, we compared our Locations to the locations of competing Cajun restaurants. There were 3 Cajun restaurants within 2 miles of Location 1, one close to downtown and the other 2 a drive south. Within 2 miles of Location 2, the closest Cajun restaurant is actually same downtown restaurant near Location 1. 
Location 2 would be farther away from competitor restaurants. While this may lose potential customers downtown, it is more accessible to the other parts of town. Perhaps people on the North, West, and South side of the city want Cajun food and do not want to drive all the way down town for that. Having the restaurant at Location 2, being the only Cajun restaurant in that area, makes Location 2 more accessible.

## Conclusion

After considering all the data, Location 2 is the optimal location to open a restaurant. Location 2 is more accessible by a majority of the city, and is still accessible to tourists as there are other trending venues nearby. Location 2 is also ideal for gaining regular customers from the city residents, as it is closer to grocery stores and schools. Location 2 is in an area without other Cajun restaurants, making it stand out against the existing Cajun restaurants which are located on only one side of the city.